# Database generation

In [1]:
import sqlite3
import pandas as pd
import random
from datetime import datetime, timedelta

# Generate a random hire date between 2018 and mid-2024
def random_hire_date(start_year=2018, end_year=2024):
    start = datetime(start_year, 1, 1)
    end   = datetime(end_year, 6, 1)
    delta = (end - start).days
    return (start + timedelta(days=random.randint(0, delta))).date()

# Define basic employee information
employee_defs = [
    ("Wei Zhang", "Sales Manager", "Sales"),
    ("Tao Huang", "Marketing Specialist", "Marketing"),
    ("Na Li", "Product Developer", "Product Development"),
    ("John Smith", "Financial Analyst", "Finance"),
    ("Alice Chen", "IT Support", "IT"),
    ("Bob Li", "Sales Associate", "Sales"),
    ("Carol Wang", "Marketing Coordinator", "Marketing"),
    ("David Liu", "Product Engineer", "Product Development"),
    ("Eva Zhou", "Accountant", "Finance"),
    ("Frank Wu", "System Administrator", "IT"),
]

# Assemble employee list with details
employees = []
for i, (name, title, dept) in enumerate(employee_defs, start=1):
    employees.append({
        "employee_id": i,
        "name": name,
        "email": f"{name.split()[0].lower()}.{name.split()[1].lower()}@example.com",
        "hire_date": random_hire_date().isoformat(),
        "job_title": title,
        "department": dept
    })

# Define a pool of activity descriptions
activity_templates = [
    "Prepared the weekly report and met with clients.",
    "Developed a new marketing strategy and coordinated with team.",
    "Designed feature prototypes and reviewed technical specs.",
    "Analyzed quarterly financial data and presented insights.",
    "Resolved critical IT tickets and performed system maintenance.",
    "Closed major sales deals and negotiated contracts.",
    "Organized a product demo and gathered user feedback.",
    "Optimized marketing campaigns and tracked KPIs.",
    "Collaborated on cross-functional projects and documented processes.",
    "Conducted system audits and updated security protocols."
]

# Generate 10 weeks of data for each employee
data = []
start_of_period = datetime(2024, 7, 1)
for emp in employees:
    for week in range(1, 11):
        data.append({
            "employee_id": emp["employee_id"],
            "week_number": week,
            "num_meetings": random.randint(0, 10),
            "total_sales_rmb": round(random.uniform(1000, 50000), 2),
            "hours_worked": round(random.uniform(30, 60), 2),
            "activities": random.choice(activity_templates),
            "department": emp["department"],
            "hire_date": emp["hire_date"],
            "email": emp["email"],
            "job_title": emp["job_title"]
        })

# Convert to DataFrame and save to SQLite
df = pd.DataFrame(data)
conn = sqlite3.connect("employee_activity.db")
df.to_sql("employee_activity", conn, if_exists="replace", index=False)
conn.close()

# Read from SQLite and export to CSV
conn = sqlite3.connect("employee_activity.db")
df = pd.read_sql("SELECT * FROM employee_activity", conn)
df.to_csv("employee_activity.csv", index=False)
conn.close()

# Preview the first few rows
print(df.head())


   employee_id  week_number  num_meetings  total_sales_rmb  hours_worked  \
0            1            1             2         10344.08         46.37   
1            1            2             0         13624.81         38.94   
2            1            3             4         41819.71         47.34   
3            1            4             4          2555.17         54.98   
4            1            5             1         18174.49         31.06   

                                          activities department   hire_date  \
0  Collaborated on cross-functional projects and ...      Sales  2020-05-01   
1  Conducted system audits and updated security p...      Sales  2020-05-01   
2    Optimized marketing campaigns and tracked KPIs.      Sales  2020-05-01   
3  Closed major sales deals and negotiated contra...      Sales  2020-05-01   
4  Closed major sales deals and negotiated contra...      Sales  2020-05-01   

                   email      job_title  
0  wei.zhang@example.com  

# Benchmark 1 : Run the 20 example queries

In [2]:
import os
import warnings
from langchain.chat_models import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent
from dotenv import load_dotenv

warnings.filterwarnings("ignore")

# Custom prompt to ensure raw SQL output without markdown formatting
custom_prefix = """You are a highly intelligent data analyst with strong SQL knowledge.
You are connected to a SQL database and your task is to answer questions by generating accurate SQL queries.
Do not explain the query unless explicitly asked. Do not use markdown formatting like ```sql or ``` in your responses.
Return only the final answer as plain text if possible.

When generating SQL:
- Only output raw SQL syntax.
- Do not wrap your query in any backticks or markdown blocks.
- Ensure the SQL syntax is compatible with SQLite.

You will be given the schema of the table before you start.
Begin!"""

# 1) Initialize connection to the SQLite database
db_uri = "sqlite:///employee_activity.db"
db = SQLDatabase.from_uri(db_uri)

# 2) Initialize the LLM (e.g., GPT-4)
llm = ChatOpenAI(model="gpt-4", temperature=0)

# 3) Create the SQL Agent with the database and LLM
agent = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    prefix=custom_prefix
)

# 4) Define a list of 20 natural language queries to test
queries = [
    "What is the email address of the employee who is the Sales Manager?",
    "Which employee in the company works in the Product Development department?",
    "What was the sales revenue of 'Wei Zhang' for the week starting on '2024-08-28'?",
    "Who are the employees working in the 'Finance' department?",
    "Retrieve the total number of meetings attended by 'Na Li' in her weekly updates.",
    "Which employees worked more than 40 hours during week 1?",
    "How many employees does the company have in total?",
    "What is the average hours worked by all employees during week 2?",
    "How much total sales revenue has the Sales department generated to date?",
    "What is the total sales revenue generated by the company during week 1?",
    "Who worked the most hours during the first week of September 2024?",
    "Which employee attended the most meetings during week 2?",
    "Which employees in the company were hired during a time of industry recession?",
    "Who are the employees that faced challenges with customer retention, and what solutions did they propose?",
    "Which employees work in roles that likely require data analysis or reporting skills?",
    "List all employees who work in the IT department within the company.",
    "Compare the hours worked by 'Wei Zhang' and 'Tao Huang' during week 1.",
    "Who are the top 3 employees by total hours worked during the last 4 weeks?",
    "Who achieved the highest sales revenue in a single week, and when?",
    "What is the total number of hours worked and average sales revenue for employees in the Business Development department?"
]

# 5) Run and print the result of each query
for idx, q in enumerate(queries, start=1):
    print(f"\n--- Query {idx} ---")
    print("Q:", q)
    try:
        answer = agent.run(q)
    except Exception as e:
        answer = f"Error: {e}"
    print("A:", answer)


--- Query 1 ---
Q: What is the email address of the employee who is the Sales Manager?


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: employee_activityThe table name provided doesn't seem to contain the information I need. I need a table that contains employee details such as their role and email address. I should check the schema of the 'employee_activity' table to confirm.
Action: sql_db_schema
Action Input: employee_activity
CREATE TABLE employee_activity (
	employee_id INTEGER, 
	week_number INTEGER, 
	num_meetings INTEGER, 
	total_sales_rmb REAL, 
	hours_worked REAL, 
	activities TEXT, 
	department TEXT, 
	hire_date TEXT, 
	email TEXT, 
	job_title TEXT
)

/*
3 rows from employee_activity table:
employee_id	week_number	num_meetings	total_sales_rmb	hours_worked	activities	department	hire_date	email	job_title
1	1	2	10344.08	46.37	Collaborated on cross-functional projects and documented processes.	Sales	2020-05-01	wei.zhang@example.com	Sales Man

### Testing llm accuracy by manually writing sql

### query 5

In [4]:
# Connect to the database
conn = sqlite3.connect("employee_activity.db")
cursor = conn.cursor()

# Query: Calculate the total number of meetings attended by Na Li
query = '''
SELECT SUM(num_meetings) AS total_meetings
FROM employee_activity
WHERE email = 'na.li@example.com';
'''

cursor.execute(query)
result = cursor.fetchone()
conn.close()

# Print the result
print("\n📊 Total Meetings Attended by Na Li:")
print(f"Total Meetings: {result[0]}")


📊 Total Meetings Attended by Na Li:
Total Meetings: 33


### query 6

In [6]:
import sqlite3

conn = sqlite3.connect("employee_activity.db")
cursor = conn.cursor()

query = '''
SELECT employee_id, hours_worked, email
FROM employee_activity
WHERE week_number = 1 AND hours_worked > 40
ORDER BY hours_worked DESC;
'''

cursor.execute(query)
results = cursor.fetchall()
conn.close()

print("📊 Employees who worked more than 40 hours in Week 1:")
for row in results:
    print(f"Employee ID: {row[0]}, Hours: {row[1]:.2f}, Email: {row[2]}")

📊 Employees who worked more than 40 hours in Week 1:
Employee ID: 3, Hours: 57.88, Email: na.li@example.com
Employee ID: 7, Hours: 51.70, Email: carol.wang@example.com
Employee ID: 4, Hours: 47.85, Email: john.smith@example.com
Employee ID: 2, Hours: 47.41, Email: tao.huang@example.com
Employee ID: 9, Hours: 47.40, Email: eva.zhou@example.com
Employee ID: 6, Hours: 47.23, Email: bob.li@example.com
Employee ID: 8, Hours: 46.49, Email: david.liu@example.com
Employee ID: 1, Hours: 46.37, Email: wei.zhang@example.com


# Benchmark 2 : improve llm performance

## Improve QUERY 13 using external knowledge

In [9]:
pip install google-search-results

Note: you may need to restart the kernel to use updated packages.


In [10]:
from langchain.agents import Tool, initialize_agent
from langchain_community.utilities.serpapi import SerpAPIWrapper 

# set API Key
os.environ["OPENAI_API_KEY"] = "sk"
os.environ["SERPAPI_API_KEY"] = "xxx"

# Custom prompt to ensure raw SQL output without markdown formatting
custom_prefix = """You are a highly intelligent data analyst with strong SQL knowledge.
You are connected to a SQL database and your task is to answer questions by generating accurate SQL queries.
Do not explain the query unless explicitly asked. Do not use markdown formatting like ```sql or ``` in your responses.
Return only the final answer as plain text if possible.

When generating SQL:
- Only output raw SQL syntax.
- Do not wrap your query in any backticks or markdown blocks.
- Ensure the SQL syntax is compatible with SQLite.

You will be given the schema of the table before you start.
Begin!"""

# initialize tools and agents
search = SerpAPIWrapper()
llm    = ChatOpenAI(model="gpt-4", temperature=0)

db     = SQLDatabase.from_uri("sqlite:///employee_activity.db")
sql_agent = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    prefix=custom_prefix
)

tools = [
    Tool(
        name="Recession Search",
        func=search.run,
        description="Find historical recession dates via SerpAPI."
    ),
    Tool(
        name="Employee Database Agent",
        func=sql_agent.run,
        description="Query the employee_activity database."
    )
]

# Initialize LangChain Multi-Tool Agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

# Run Query: Combine external recession info + internal hiring data
print(agent.run("Which employees were hired during the most recent recession?"))



> Entering new AgentExecutor chain...
First, I need to find out when the most recent recession was.
Action: Recession Search
Action Input: {"query": "most recent recession"}
Observation: ['Search query. Search the web. Skip to main. News ... Since 1929, there have been 14 recessionary periods in the U.S. The most recent recession ...', 'Search query. Tip: Try a valid symbol or a specific company name ... The most recent recession was in the spring of 2020 when the COVID ...', 'While the user query explicitly asks for “forecasting” tricks, the ... Since the end of the most recent recession in April 2020, Real ...', 'Although most of the economies around the globe are seemingly doing quite well since the most recent recession of 2018, there is always more ...', 'Search query.. Main navigation. ABOUT. Who We Are · State Policy ... The most recent recession was more severe — deeper and longer — than ...', 'The most recent recession was preceded ... The primary new data source discussed i

## Improve QUERY 14 by updating db and prompt

In [12]:
import sqlite3
import pandas as pd
import random
from datetime import datetime, timedelta

# Generate a random hire date between 2018 and mid-2024
def random_hire_date(start_year=2018, end_year=2024):
    start = datetime(start_year, 1, 1)
    end   = datetime(end_year, 6, 1)
    delta = (end - start).days
    return (start + timedelta(days=random.randint(0, delta))).date()

# Define basic employee information
employee_defs = [
    ("Wei Zhang", "Sales Manager", "Sales"),
    ("Tao Huang", "Marketing Specialist", "Marketing"),
    ("Na Li", "Product Developer", "Product Development"),
    ("John Smith", "Financial Analyst", "Finance"),
    ("Alice Chen", "IT Support", "IT"),
    ("Bob Li", "Sales Associate", "Sales"),
    ("Carol Wang", "Marketing Coordinator", "Marketing"),
    ("David Liu", "Product Engineer", "Product Development"),
    ("Eva Zhou", "Accountant", "Finance"),
    ("Frank Wu", "System Administrator", "IT"),
]

# Assemble employee list with details
employees = []
for i, (name, title, dept) in enumerate(employee_defs, start=1):
    employees.append({
        "employee_id": i,
        "name": name,
        "email": f"{name.split()[0].lower()}.{name.split()[1].lower()}@example.com",
        "hire_date": random_hire_date().isoformat(),
        "job_title": title,
        "department": dept
    })

activity_templates = [
    # General business activities
    "Prepared the weekly report and met with clients.",
    "Developed a new marketing strategy and coordinated with team.",
    "Designed feature prototypes and reviewed technical specs.",
    "Analyzed quarterly financial data and presented insights.",
    "Resolved critical IT tickets and performed system maintenance.",
    "Closed major sales deals and negotiated contracts.",
    "Organized a product demo and gathered user feedback.",
    "Optimized marketing campaigns and tracked KPIs.",
    "Collaborated on cross-functional projects and documented processes.",
    "Conducted system audits and updated security protocols.",
    "Faced challenges with customer retention; launched loyalty program to improve retention."
]

# Generate 10 weeks of data for each employee
data = []
start_of_period = datetime(2024, 7, 1)
for emp in employees:
    for week in range(1, 11):
        data.append({
            "employee_id": emp["employee_id"],
            "week_number": week,
            "num_meetings": random.randint(0, 10),
            "total_sales_rmb": round(random.uniform(1000, 50000), 2),
            "hours_worked": round(random.uniform(30, 60), 2),
            "activities": random.choice(activity_templates),
            "department": emp["department"],
            "hire_date": emp["hire_date"],
            "email": emp["email"],
            "job_title": emp["job_title"]
        })

# Convert to DataFrame and save to SQLite
df = pd.DataFrame(data)
conn = sqlite3.connect("employee_activity.db")
df.to_sql("employee_activity", conn, if_exists="replace", index=False)
conn.close()

# Read from SQLite and export to CSV
conn = sqlite3.connect("employee_activity.db")
df = pd.read_sql("SELECT * FROM employee_activity", conn)
df.to_csv("employee_activity.csv", index=False)
conn.close()

# Preview the first few rows
print(df.head())


   employee_id  week_number  num_meetings  total_sales_rmb  hours_worked  \
0            1            1             6         27983.18         30.57   
1            1            2             7         18662.31         50.92   
2            1            3             1          3347.44         53.31   
3            1            4             8         12112.57         45.99   
4            1            5             9         42508.38         35.81   

                                          activities department   hire_date  \
0  Resolved critical IT tickets and performed sys...      Sales  2022-11-30   
1    Optimized marketing campaigns and tracked KPIs.      Sales  2022-11-30   
2   Prepared the weekly report and met with clients.      Sales  2022-11-30   
3  Resolved critical IT tickets and performed sys...      Sales  2022-11-30   
4    Optimized marketing campaigns and tracked KPIs.      Sales  2022-11-30   

                   email      job_title  
0  wei.zhang@example.com  

In [14]:
warnings.filterwarnings("ignore")

custom_prefix = """You are a highly intelligent data analyst with strong SQL knowledge.
You are connected to a SQL database and your task is to answer questions by generating accurate SQL queries.
Do not explain the query unless explicitly asked. Do not use markdown formatting like ```sql or ``` in your responses.
Return only the final answer as plain text if possible.

When generating SQL:
- Only output raw SQL syntax.
- Do not wrap your query in any backticks or markdown blocks.
- Ensure the SQL syntax is compatible with SQLite.

You will be given the schema of the table before you start.
Begin!"""


# 1) Initialize connection to the SQLite database
db_uri = "sqlite:///employee_activity.db"
db = SQLDatabase.from_uri(db_uri)

# 2) Initialize the LLM (e.g., GPT-4)
llm = ChatOpenAI(model="gpt-4", temperature=0)

# 3) Create the SQL Agent with the database and LLM
agent = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    prefix=custom_prefix
)

# 4) Define a list of 20 natural language queries to test
queries = [
    "Who are the employees that faced challenges with customer retention, and what solutions did they propose?"
]

# 5) Run and print the result of each query
for idx, q in enumerate(queries, start=1):
    print(f"\n--- Query {idx} ---")
    print("Q:", q)
    try:
        answer = agent.run(q)
    except Exception as e:
        answer = f"Error: {e}"
    print("A:", answer)


--- Query 1 ---
Q: Who are the employees that faced challenges with customer retention, and what solutions did they propose?


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: employee_activityThe 'employee_activity' table seems to be the most relevant one for this query. I should check its schema to understand the structure of the data.
Action: sql_db_schema
Action Input: employee_activity
CREATE TABLE employee_activity (
	employee_id INTEGER, 
	week_number INTEGER, 
	num_meetings INTEGER, 
	total_sales_rmb REAL, 
	hours_worked REAL, 
	activities TEXT, 
	department TEXT, 
	hire_date TEXT, 
	email TEXT, 
	job_title TEXT
)

/*
3 rows from employee_activity table:
employee_id	week_number	num_meetings	total_sales_rmb	hours_worked	activities	department	hire_date	email	job_title
1	1	6	27983.18	30.57	Resolved critical IT tickets and performed system maintenance.	Sales	2022-11-30	wei.zhang@example.com	Sales Manager
1	2	7	18662.31	50.92	Optimized marketing c

In [13]:
warnings.filterwarnings("ignore")

# Custom prompt to ensure raw SQL output without markdown formatting
custom_prefix = """You are a highly intelligent data analyst with strong SQL knowledge.
You are connected to a SQL database and your task is to answer questions by generating accurate SQL queries.
When a question has multiple parts (e.g., asking both "who" and "what"), provide a clear and structured answer with headings like:
- "Employees involved"
- "Solutions proposed"

Do not explain the SQL query unless explicitly asked.
Do not use markdown formatting like ```sql or ``` in your responses.
When listing employees, always use SELECT DISTINCT so that each employee appears only once.
Return only the final answer as plain text if possible.

Ensure the SQL syntax is compatible with SQLite.

Begin!
"""


# 1) Initialize connection to the SQLite database
db_uri = "sqlite:///employee_activity.db"
db = SQLDatabase.from_uri(db_uri)

# 2) Initialize the LLM (e.g., GPT-4)
llm = ChatOpenAI(model="gpt-4", temperature=0)

# 3) Create the SQL Agent with the database and LLM
agent = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    prefix=custom_prefix
)

queries = [
    "Who are the employees that faced challenges with customer retention, and what solutions did they propose?"
]

# 5) Run and print the result of each query
for idx, q in enumerate(queries, start=1):
    print(f"\n--- Query {idx} ---")
    print("Q:", q)
    try:
        answer = agent.run(q)
    except Exception as e:
        answer = f"Error: {e}"
    print("A:", answer)


--- Query 1 ---
Q: Who are the employees that faced challenges with customer retention, and what solutions did they propose?


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""employee_activityThe 'employee_activity' table seems relevant to the question. I should check its schema to understand its structure and the data it contains.
Action: sql_db_schema
Action Input: "employee_activity"
CREATE TABLE employee_activity (
	employee_id INTEGER, 
	week_number INTEGER, 
	num_meetings INTEGER, 
	total_sales_rmb REAL, 
	hours_worked REAL, 
	activities TEXT, 
	department TEXT, 
	hire_date TEXT, 
	email TEXT, 
	job_title TEXT
)

/*
3 rows from employee_activity table:
employee_id	week_number	num_meetings	total_sales_rmb	hours_worked	activities	department	hire_date	email	job_title
1	1	6	27983.18	30.57	Resolved critical IT tickets and performed system maintenance.	Sales	2022-11-30	wei.zhang@example.com	Sales Manager
1	2	7	18662.31	50.92	Optimized marketing ca